```
--- Day 7: Internet Protocol Version 7 ---

While snooping around the local network of EBHQ, you compile a list of IP addresses (they're IPv7, of course; IPv6 is much too limited). You'd like to figure out which IPs support TLS (transport-layer snooping).

An IP supports TLS if it has an Autonomous Bridge Bypass Annotation, or ABBA. An ABBA is any four-character sequence which consists of a pair of two different characters followed by the reverse of that pair, such as xyyx or abba. However, the IP also must not have an ABBA within any hypernet sequences, which are contained by square brackets.

For example:

abba[mnop]qrst supports TLS (abba outside square brackets).
abcd[bddb]xyyx does not support TLS (bddb is within square brackets, even though xyyx is outside square brackets).
aaaa[qwer]tyui does not support TLS (aaaa is invalid; the interior characters must be different).
ioxxoj[asdfgh]zxcvbn supports TLS (oxxo is outside square brackets, even though it's within a larger string).
How many IPs in your puzzle input support TLS?

```

In [109]:
import re

HYPERNET = re.compile("\[([a-z]*)\]")

def check_abba(ip):
    
    for a,b,c,d in zip(ip[:-3], ip[1:-2], ip[2:-1], ip[3:]):
        if a==d and b==c and a!=b:
            return True
        
    return False

def check_tls(ip): 
    
    ip = ip.strip()    
    for hypernet in HYPERNET.findall(ip):
        if check_abba(hypernet):
            return False
        
    for segment in HYPERNET.sub("#", ip).split('#'):
        if check_abba(segment):
            return True
            
    return False

In [107]:
check_tls('abba[mnop]qrst[sdfdf]kjhjk')

True

In [105]:
check_tls('abcd[bddb]xyyx')

False

In [112]:
check_tls('aaa[qwer]tyui')

False

In [113]:
check_tls('ioxxoj[asdfgh]zxcvbn')

True

In [114]:
with open('inputs/day7.txt', 'rt') as fd:
    print(sum(1 for ip in filter(check_tls, fd)))

115


```
--- Part Two ---

You would also like to know which IPs support SSL (super-secret listening).

An IP supports SSL if it has an Area-Broadcast Accessor, or ABA, anywhere in the supernet sequences (outside any square bracketed sections), and a corresponding Byte Allocation Block, or BAB, anywhere in the hypernet sequences. An ABA is any three-character sequence which consists of the same character twice with a different character between them, such as xyx or aba. A corresponding BAB is the same characters but in reversed positions: yxy and bab, respectively.

For example:

aba[bab]xyz supports SSL (aba outside square brackets with corresponding bab within square brackets).
xyx[xyx]xyx does not support SSL (xyx, but no corresponding yxy).
aaa[kek]eke supports SSL (eke in supernet with corresponding kek in hypernet; the aaa sequence is not related, because the interior character must be different).
zazbz[bzb]cdb supports SSL (zaz has no corresponding aza, but zbz has a corresponding bzb, even though zaz and zbz overlap).
How many IPs in your puzzle input support SSL?
```

In [119]:
def find_aba(ip):    
    for a,b,c in zip(ip[:-2], ip[1:-1], ip[2:]):
        if a==c:
            yield a + b + a, b + a + b  
            
def check_ssl(ip):   
    
    ip = ip.strip()    
    hypernets = HYPERNET.findall(ip)        
    for segment in HYPERNET.sub("#", ip).split('#'):
        for aba, bab in find_aba(segment):
            if len([h for h in hypernets if bab in h]) > 0:
                return True
            
    return False    

In [120]:
check_ssl("aba[bab]xyz")

True

In [121]:
check_ssl("xyx[xyx]xyx")

False

In [122]:
check_ssl("aaa[kek]eke")

True

In [123]:
check_ssl("zazbz[bzb]cdb")

True

In [124]:
with open('inputs/day7.txt', 'rt') as fd:
    print(sum(1 for ip in filter(check_ssl, fd)))

231
